In [1]:
# content-based filtering, 
# where content of the movie (overview, cast, crew, keyword, tagline etc) is used to find similarities with other movies. Then the movies that are most likely to be similar are recommended.
# Credits, Genres & Keywords based recommender (Has better metadata: the top 3 actors, the director, related genres & movie plot-keywords)

In [2]:
# importing python libraries

import numpy as np # linear algebra
import pandas as pd # data-processing, CSV-file I/O (e.g pd.read_csv)

# Loading the data

df1 = pd.read_csv('tmdb_5000_credits.csv',delimiter=',')
df2 = pd.read_csv('tmdb_5000_movies.csv',delimiter=',')

In [3]:
df1.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
df2.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
movie_id    4803 non-null int64
title       4803 non-null object
cast        4803 non-null object
crew        4803 non-null object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [7]:
df1.shape

(4803, 4)

In [8]:
df2.shape

(4803, 20)

In [9]:
# joining the two datasets based on the 'id' column

df1.columns = ['id','tittle','cast','crew']
df2 = df2.merge(df1,on='id')

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
id        4803 non-null int64
tittle    4803 non-null object
cast      4803 non-null object
crew      4803 non-null object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [11]:
df2.info() #here, number of column has increased from 20 to 23 ('title','cast','crew' added from df1 to df2) due to merging on the 'id' column

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 23 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [12]:
# Since the features are currently in the form of "stringified" lists, converting them into their corresponding python objects
# from Abstract Syntax Tree (ast) module
# By importing literal_eval function for safely evaluating strings containing Python expressions from untrusted sources without the need to parse the values oneself.
# The string or node provided may only consist of the following Python literal structures: strings, numbers, tuples, lists, dicts, booleans, and None.


from ast import literal_eval

# Specifying the desired features list and applying literal_eval function to it

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [13]:
df2[feature]

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
1       [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
5       [{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...
6       [{'id': 16, 'name': 'Animation'}, {'id': 10751...
7       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
8       [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
9       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
10      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
11      [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
12      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
13      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
14      [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
15      [{'id': 12, 'name': 'Adventure'}, {'id': 10751...
16      [{'id': 878, 'name': 'Science Fiction'}, {'id'...
17      [{'id'

In [14]:
# Feature Extraction:
# Some functions to extract desired information from each feature
# From the "cast", "crew" & "keywords" features, 
# Need to extract ""the three most important actors"", ""the director"", ""the keywords associated with that movie"" 

# 1. Get director's name from 'crew' feature.
# if director is not listed, return NaN

def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

# 2. Getting list top 3 elements or, entire list if elements < 3 (whichever is more)
# Specifying the desired features list and applying literal_eval function to it
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3: # checking whether more than 3 elements exist 
            names = names[:3] # & if yes, then returning only the 1st 3 ekements. else, returning the entire list
        return names
    return [] # returning empty list incase of missing or malformed data

In [15]:
# Defining new 'director', 'cast', 'genres' & 'keywords' features that are in a suitable form for manipulation

df2['director'] = df2['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [16]:
# printing the new features of the first 10 films

df2[['title', 'cast', 'director', 'keywords', 'genres']].head(10)

,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton,"[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"
5,Spider-Man 3,"[Tobey Maguire, Kirsten Dunst, James Franco]",Sam Raimi,"[dual identity, amnesia, sandstorm]","[Fantasy, Action, Adventure]"
6,Tangled,"[Zachary Levi, Mandy Moore, Donna Murphy]",Byron Howard,"[hostage, magic, horse]","[Animation, Family]"
7,Avengers: Age of Ultron,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...",Joss Whedon,"[marvel comic, sequel, superhero]","[Action, Adventure, Science Fiction]"
8,Harry Potter and the Half-Blood Prince,"[Daniel Radcliffe, Rupert Grint, Emma Watson]",David Yates,"[witch, magic, broom]","[Adventure, Fantasy, Family]"
9,Batman v Superman: Dawn of Justice,"[Ben Affleck, Henry Cavill, Gal Gadot]",Zack Snyder,"[dc comics, vigilante, superhero]","[Action, Adventure, Fantasy]"


In [17]:
# Cleaning data:
# Converting 'names' & 'keywords' instances into lowercase &
# Stripping any space between the names
# So that the vectorizer (CountVectorizer) doesn't count "Jonny Depp" & "Jonny Galecki" as the same object

# Function to convert all strings to lower case & strip names of any space

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
    # checking whether director exists. If not, returning empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ",""))
        else:
            return ''


In [18]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [19]:
# Joining the keywords, cast, director, & genres of each movie instance & adding them as ‘soup’ column in dataframe 

def create_soup(x): # minimalistic corpus of text documents
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [20]:
# Import CountVectorizer and create the count matrix
# CountVectorizer converts a collection of text documents to a matrix of token counts.
# It implements both tokenization and occurrence counting in a single class

from sklearn.feature_extraction.text import CountVectorizer

# a built-in stop word list for English is used. Here, Stop words are words like “and”, “the”, “him”, 
# which are presumed to be uninformative in representing the content of a text, 
# and which may be removed to avoid them being construed as signal for prediction

count = CountVectorizer(stop_words='english')

# tokenize and count the word occurrences of a minimalistic corpus of text documents, here, ‘soup’

count_matrix = count.fit_transform(df2['soup'])

In [21]:
count_matrix

<4803x11520 sparse matrix of type '<class 'numpy.int64'>'
	with 42935 stored elements in Compressed Sparse Row format>

In [22]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Cosine similarity is a method to measure the difference between two non-zero vectors of an inner product space

cosine_sim = cosine_similarity(count_matrix, count_matrix) # the output will be the pair-wise similarities between all samples in count_matrix

In [23]:
# Reset index of our main DataFrame and construct reverse mapping
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [24]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Getting the index of the movie that matches the title
    idx = indices[title]

    # Getting the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sorting the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Getting the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Getting the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Returning the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [25]:
# Calling function that will show top 10 best-recommended movies on the given input
get_recommendations('The Dark Knight Rises', cosine_sim)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

In [26]:
get_recommendations('John Carter', cosine_sim)

183     The Hunger Games: Catching Fire
260                        Ender's Game
270                         The Martian
400                           Divergent
2444                    Damnation Alley
4401                The Helix... Loaded
83                           The Lovers
193                         After Earth
4117                 Six-String Samurai
91         Independence Day: Resurgence
Name: title, dtype: object